In [1]:
import torch
import numpy as np
from soft_dp import SoftSODA

In [2]:
gt = np.array([[5,10], [11,19], [28,32]])
proposals = np.array([[3,7], [4,14], [15,33],[24,30],[25,33],[37,45]])

In [3]:
def box_iou(boxes1, boxes2):
    area1 = boxes1[:, 1] - boxes1[:, 0]
    area2 = boxes2[:, 1] - boxes2[:, 0]
    lt = torch.max(boxes1[:, None, 0], boxes2[:, 0])  # [N,M,2]
    rb = torch.min(boxes1[:, None, 1], boxes2[:, 1])  # [N,M,2]
    inter = (rb - lt).clamp(min=0)  # [N,M,2]
    union = area1[:, None] + area2 - inter
    iou = inter / (union + 1e-5)
    return iou, union


def generalized_box_iou(boxes1, boxes2):
    """
    Generalized IoU from https://giou.stanford.edu/

    The boxes should be in [x0, y0, x1, y1] format

    Returns a [N, M] pairwise matrix, where N = len(boxes1)
    and M = len(boxes2)
    """
    #print(boxes1.size(), boxes2.size())
    # degenerate boxes gives inf / nan results
    # so do an early check
    assert (boxes1[:, 1:] >= boxes1[:, :1]).all()
    assert (boxes2[:, 1:] >= boxes2[:, :1]).all()
    iou, union = box_iou(boxes1, boxes2)
    lt = torch.min(boxes1[:, None, 0], boxes2[:, 0])
    rb = torch.max(boxes1[:, None, 1], boxes2[:, 1])
    area = (rb - lt).clamp(min=0)  # [N,M,2]
    giou = iou - (area - union) / (area + 1e-5)
    return giou, iou

In [6]:
giou_cost, iou_cost = generalized_box_iou(torch.from_numpy(gt), torch.from_numpy(proposals))
giou_cost.unsqueeze(0)

tensor([[[ 0.2857,  0.5000, -0.1786, -0.5600, -0.5357, -0.6750],
         [-0.2500,  0.2000,  0.1818, -0.2632, -0.2727, -0.5294],
         [-0.7241, -0.5000,  0.2222,  0.2500,  0.5000, -0.2941]]])

In [8]:
loss, soda_matrix = SoftSODA(-giou_cost.unsqueeze(0), gamma_min=0.01)
loss /3

tensor(-0.3939, device='cuda:0')

In [14]:
# torch.cat(soda_matrix.vars, 0)

In [10]:
soda_matrix[1:, 1:]

array([[2., 2., 1., 1., 1., 1.],
       [0., 0., 2., 1., 1., 1.],
       [0., 0., 2., 2., 2., 1.]])

In [11]:
1-giou_cost

tensor([[0.7143, 0.5000, 1.1786, 1.5600, 1.5357, 1.6750],
        [1.2500, 0.8000, 0.8182, 1.2632, 1.2727, 1.5294],
        [1.7241, 1.5000, 0.7778, 0.7500, 0.5000, 1.2941]])

In [12]:
-giou_cost

tensor([[-0.2857, -0.5000,  0.1786,  0.5600,  0.5357,  0.6750],
        [ 0.2500, -0.2000, -0.1818,  0.2632,  0.2727,  0.5294],
        [ 0.7241,  0.5000, -0.2222, -0.2500, -0.5000,  0.2941]])